In [1]:
import os
from pymongo import MongoClient

In [2]:
def get_directories(path: str):
    """
    Get all directory names under the specified path.
    
    :param path: The root directory path to search.
    :return: A list of directory names.
    """
    try:
        return [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
    except FileNotFoundError:
        print(f"Error: The path '{path}' does not exist.")
        return []
    except PermissionError:
        print(f"Error: Permission denied for path '{path}'.")
        return []
def count_files(directory_path: str) -> int:
    """
    Count the number of files in a given directory.

    :param directory_path: The directory path to search.
    :return: The number of files in the directory.
    """
    try:
        return sum(1 for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f)))
    except FileNotFoundError:
        print(f"Error: The directory '{directory_path}' does not exist.")
        return 0
    except PermissionError:
        print(f"Error: Permission denied for directory '{directory_path}'.")
        return 0
    except Exception as e:
        print(f"Error: {e}")
        return 0
def connect_mongodb(uri,db_name, collection_name):
    # Connect to MongoDB 
    client = MongoClient(uri)
    
    # Select the database and collection
    db = client[db_name]
    collection = db[collection_name]
    
    return collection
def update_mongo_fields(collection, query_filter, update_fields):
    """
    Update one or more fields in a MongoDB collection.

    Parameters:
    - collection: The MongoDB collection object.
    - query_filter: A dictionary specifying the filter to select documents.
    - update_fields: A dictionary of fields to be updated, with keys as field names and values as the new values.

    Example:
    update_mongo_fields(
        collection=your_collection,
        query_filter={'_id': 'in.codeseed.audify'},
        update_fields={'apkfile_name': 'new_name.apk', 'extract_manifest': ['UPDATED', 'MODIFIED']}
    )
    """
    # Print query filter for debugging
    print(f"Query filter: {query_filter}")

    # Prepare the update object with $set to update only the specified fields
    # Check if value is string or list, update accordingly
    update_query = {"$set": {}}

    for field, value in update_fields.items():
        if isinstance(value, list):
            update_query["$set"][field] = value  # If it's a list, set the entire list
        else:
            update_query["$set"][field] = value  # If it's a string, update as a string

    # Perform the update in the collection
    result = collection.update_many(query_filter, update_query)

    # Check if anything was matched or updated
    if result.matched_count == 0:
        print("No documents matched the filter.")
    elif result.modified_count == 0 and result.matched_count > 0:
        print("Documents matched but none were updated (maybe no fields were changed).")
    
    # Print out the result of the update
    print(f"Matched {result.matched_count} document(s), updated {result.modified_count} document(s).")

In [3]:
# root_traffic_directory_path = r"E:\wearable-capture-traffic\wearable-standalone"
root_traffic_directory_path = r"E:\wearable-capture-traffic\wearable-app"

In [4]:
mongoDB_uri = 'mongodb://192.168.1.14:27017'
mongoDB_database = 'wearable-project' 
# mongoDB_collection = 'wearable-standalone'
mongoDB_collection = 'wearable-app'

In [5]:
array_traffic_directory = get_directories(root_traffic_directory_path)
print(array_traffic_directory)

['adarshurs.android.vlcmobileremote', 'app.EnForma.fitcoapp.net', 'app.entrenamientogenetico.android.fitcoapp.net', 'app.f4square.fitness', 'app.fit.meet', 'app.growwithjo.diet.fitness', 'app.krowdfit', 'app.LomasGym.fitcoapp.net', 'app.mfit.personal', 'app.midfit.train.own', 'app.mlfitness.diet.workout', 'app.MOVE.fitcoapp.net', 'app.weightfit.android', 'appnativo.navysportcenter.fitcoapp.net', 'at.radioplayer.android', 'biz.mobidev.mdplayer', 'blacknote.amazfitmaster', 'br.com.fitastic.appaluno', 'br.com.mufitness.station.aluno', 'br.com.personalfitapp', 'br.com.tecnofit.mobileapp', 'br.com.w12.bluefit', 'br.com.w12.heyfit', 'br.com.w12.redfit', 'br.com.w12.skyfitapp', 'calorie.pedo.fitbit', 'club.nfit', 'club.skllzz.fitness', 'clubwise.com.fitsense', 'cn.fitdays.fitdays', 'co.macrofit.macroFit', 'coach.zing.fitness', 'com.abhijitvalluri.android.fitnotifications', 'com.abyx.fit.connect', 'com.acmeaom.android.myradar', 'com.acquasys.contrack', 'com.actofitSmartScale', 'com.adi.remote.

In [6]:
# Connect to the MongoDB collection
collection = connect_mongodb(mongoDB_uri,mongoDB_database,mongoDB_collection)

In [7]:
for i in range(len(array_traffic_directory)):
    print("--------------------- Loop-"+str(i)+"---------------------")
    package_name = array_traffic_directory[i]
    print("package_name: ",package_name)
    traffic_directory = root_traffic_directory_path+"\\"+package_name
    print("traffic directory: ",traffic_directory)
    file_number = count_files(traffic_directory)
    print("file_number: ",file_number)
    if(file_number==0):
        update_mongo_fields(
                    collection, 
                    {'_id': str(package_name)}, 
                    {'sent-out': 'no data'}
                )
#     break

--------------------- Loop-0---------------------
package_name:  adarshurs.android.vlcmobileremote
traffic directory:  E:\wearable-capture-traffic\wearable-app\adarshurs.android.vlcmobileremote
file_number:  11
--------------------- Loop-1---------------------
package_name:  app.EnForma.fitcoapp.net
traffic directory:  E:\wearable-capture-traffic\wearable-app\app.EnForma.fitcoapp.net
file_number:  7
--------------------- Loop-2---------------------
package_name:  app.entrenamientogenetico.android.fitcoapp.net
traffic directory:  E:\wearable-capture-traffic\wearable-app\app.entrenamientogenetico.android.fitcoapp.net
file_number:  0
Query filter: {'_id': 'app.entrenamientogenetico.android.fitcoapp.net'}
Matched 1 document(s), updated 1 document(s).
--------------------- Loop-3---------------------
package_name:  app.f4square.fitness
traffic directory:  E:\wearable-capture-traffic\wearable-app\app.f4square.fitness
file_number:  22
--------------------- Loop-4---------------------
package_

Matched 1 document(s), updated 1 document(s).
--------------------- Loop-145---------------------
package_name:  com.fitlab.spartan
traffic directory:  E:\wearable-capture-traffic\wearable-app\com.fitlab.spartan
file_number:  5
--------------------- Loop-146---------------------
package_name:  com.fitlapapp
traffic directory:  E:\wearable-capture-traffic\wearable-app\com.fitlapapp
file_number:  3
--------------------- Loop-147---------------------
package_name:  com.fitlog.fitlog
traffic directory:  E:\wearable-capture-traffic\wearable-app\com.fitlog.fitlog
file_number:  2
--------------------- Loop-148---------------------
package_name:  com.fitmama
traffic directory:  E:\wearable-capture-traffic\wearable-app\com.fitmama
file_number:  2
--------------------- Loop-149---------------------
package_name:  com.fitmind
traffic directory:  E:\wearable-capture-traffic\wearable-app\com.fitmind
file_number:  3
--------------------- Loop-150---------------------
package_name:  com.fitness.hiit.

file_number:  11
--------------------- Loop-404---------------------
package_name:  com.squats.fittr
traffic directory:  E:\wearable-capture-traffic\wearable-app\com.squats.fittr
file_number:  28
--------------------- Loop-405---------------------
package_name:  com.starmax.runmefit
traffic directory:  E:\wearable-capture-traffic\wearable-app\com.starmax.runmefit
file_number:  4
--------------------- Loop-406---------------------
package_name:  com.std.fitness.point
traffic directory:  E:\wearable-capture-traffic\wearable-app\com.std.fitness.point
file_number:  5
--------------------- Loop-407---------------------
package_name:  com.step.walking.fit
traffic directory:  E:\wearable-capture-traffic\wearable-app\com.step.walking.fit
file_number:  20
--------------------- Loop-408---------------------
package_name:  com.steppi.steppifitness
traffic directory:  E:\wearable-capture-traffic\wearable-app\com.steppi.steppifitness
file_number:  11
--------------------- Loop-409------------------